# Module 1 - Key Sustainability Initiatives (KSIs) FROM Companies' Annual reports

#### 1. Extract Companies list using :
   * FINLAND Companies List: 
      - [ValueToday](https://www.value.today/headquarters/finland)
      - [Wikipedia](https://en.wikipedia.org/wiki/List_of_largest_companies_in_Finland)
      - [Nordic Market Data AB](https://largestcompanies.com/toplists/finland/largest-companies-by-turnover)
   * GLOBAL Companies List:
      - [Reports (2020, 2021) evaluated by PwC with Bloomberg](https://github.com/prak112/esg-profile/blob/4bebecd15332ae550aca1627835b451ce2bdab4f/dataset)
 <br></br>
#### 2. Refine tabular/csv data from sources and build dataframe with columns : 
   * Rank
   * Company_Name
   * Location
   * Sector
<br></br>
#### 3. Web Search (**Try Yahoo!Finance**) for each Company's Annual Report/Annual Sustainability Report
   * **If available, Save Report on local directory**
      *  Label Folder with _"Ranking-CompanyName"_
   * **If NA**, mark companies KSI data as NA or null **(for probable interpretation as - ESG proxy data/outsourced ESG activities)**
-----------
-----------

In [26]:
# Import neccessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Extraction libraries

! pip install PyPDF2 --quiet
print("Installed PyPDF2")
import PyPDF2
import re

Installed PyPDF2


In [27]:
# Extract PDF data
pdf_file = open(r"D:\GitHub_Projects\esg-profile\dataset\global-top-100-companies-june2020.pdf", 'rb')
pdf_data = PyPDF2.PdfFileReader(pdf_file)
print("Total Pages of Report:", pdf_data.numPages)

# Extract tables from pdf_data
pages = [10, 11, 12, 13, 14]
table_data = []

for page in pages:
    table = pdf_data.getPage(page).extractText()
    table_data.append(table)

print(table_data[:1])

#' N/A \n' ---'space CAPSletter character CAPSletter space nextline'-->to omit
ranks = list(range(1, 101))
company_data = []

for table in list(range(0,5)):
    regex = re.findall('[A-Z\s\W?a-z]+(?:\s)+(?:\n)+', table_data[table]) # regex and filter_data avoid all formatting-space. 
    filter_data = [ele for ele in regex if ele != ' ' and ' \n' and ',' and '-']
    if table == 0:
        company_data.append(filter_data[2:])
    else:
        company_data.append(filter_data[4:])

Total Pages of Report: 17
['PwC | Global Top 100 companies by market capitalisation 11Top 100 global companies 1-20\nSource: Bloomberg with PwC analysis30 June 2020 31 March 2020 31 Dec 2019 RankCompany name LocationSector Rank +/-(VS Mar 2020) Rank +/-(VS Dec 2019) Market  capitalisation ($bn)Rank Market capitalisation ($bn)Rank Market capitalisation ($bn) 1SAUDI ARABIAN OI Saudi Arabia Oil & Gas 0 0 1,741 1 1,602 1 1,879 2APPLE INC United States Technology 1 0 1,568 3 1,113 2 1,305 3MICROSOFT CORP United States Technology -1 0 1,505 2 1,200 3 1,203 4AMAZON.COM INC United States Consumer Services 0 1 1,337 4 971 5 916 5ALPHABET INC-A United States Technology 0 -1 953 5 799 4 923 6FACEBOOK INC-A United States Technology 1 0 629 7 475 6 585 7TENCENT Mainland China Technology 1 2 599 8 469 9 461 8ALIBABA GRP-ADR Mainland China Consumer Services -2 -1 577 6 522 7 569 9BERKSHIRE HATH -A United States Financials 0 -1 430 9 443 8 554 10VISA INC-CLASS A United States Financials 2 2 372 12 316

In [28]:
company_data[2]

[]

In [29]:
top1to20 = company_data[0]
top20to40 = company_data[1]
top40to60 = company_data[2]
top60to80 = company_data[3]
top80to100 = company_data[4]

-----------------
-----------------

### [*PROGRESS So Far:*]
* > *data_dict* output clear with **top1to20, top60to80 & top80to100** , i.e.,

    > *All elements are equally distributed amongst columns :  True*
* > ERROR in **top20to40 & top40t060** : 
    - Exchange _'CCB-H Mainland China'_ in *Company_Name* to:
    - 'CCB-H' in *Company_Name*
    - 'China' in *Location*
    - Change _'NIKE INC \n-CL B \nUnited States'_ to _'NIKE INC-CL B \nUnited States'_
* > ERROR in **top60to80** :
    * Company Class (*-A, -B, -H..*) to be filtered out/combined with *Company_Name*

--------------
--------------

In [30]:
# to find Python documenation
?str.find

Docstring:
S.find(sub[, start[, end]]) -> int

Return the lowest index in S where substring sub is found,
such that sub is contained within S[start:end].  Optional
arguments start and end are interpreted as in slice notation.

Return -1 on failure.
Type:      method_descriptor


In [31]:
# Dictionary (data_dict) to allocate all columns with precise data and load into DataFrame

data_dict = {}

data = top40to60
company_name = []
location = []
sector = []

for ele in data:
    subs = re.sub('N/A', '', ele).strip()   # substitute N/A with blanks  
    subs = re.sub('\n-REG', '-REG', ele).strip()    # substitute \n-A with -A\n,  substitute \n-REG with -REG\n  
    splits = subs.split('\n')    # split into groups by \n
    stripped = [split.strip() for split in splits if split != 'N/A' and '']     # split & filter elements
    
    try:    # to prevent any indexing errors
        # Collect Company_Name as a list --> Add as Value to 'Company_Name' Key in data_dict
        if stripped[0].find('Mainland China') == -1:
            company_name.append(stripped[0])
        else:
            stripped[0].replace('Mainland China','China')
            company_name.append(stripped[0])
        
        # Collect Location as a list --> Add as Value to 'Location' Key in data_dict
        if stripped[1] != '-A Mainland China':
            location.append(stripped[1])
        else:
            location.append(stripped[1].replace('-A Mainland China','China'))

        # Collect Sector as a list --> Add as Value to 'Sector' Key in data_dict
        sector.append(stripped[2])

    except IndexError:
        continue


# assign keys and values to data_dict
data_dict['Rank'] = ranks[40:60]
data_dict['Company_Name'] = company_name
data_dict['Location'] = location
data_dict['Sector'] = sector

#stripped
for key, val in data_dict.items():
    print(key ,':', val,'\n')

Rank : [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60] 

Company_Name : [] 

Location : [] 

Sector : [] 



-----------
-----------
### [Progress So Far]
- Corresponding errors in each column for the lack of 2 values in "Sector" column : 
    - **"Company_Name"** : SPLIT & MOVE TO Location    - 'SAP SE Germany', 'ABC-H Mainland China'
    - **"Location"**     : MOVE TO Sector              - 'Technology', 'Financials', 
                           REMOVE                      - '- CL B', 
                           SPLIT                       - '-B Denmark'
    - **"Sector"**       : MOVE TO Location            - 'United States'

- If possible correct errors 'try' block in 'data_dict' building cell - subs, and use RegEX
----------
----------

In [32]:
print("All elements are equally distributed amongst columns : ", len(ranks[20:40])==len(company_name)==len(location)==len(sector))
print(len(ranks[20:40]))
print(len(company_name))
print(len(location))
print(len(sector))


All elements are equally distributed amongst columns :  False
20
0
0
0


In [33]:

def trim_data(company_data):
    """
    substitute tags, strip spaces & split string into elements
    
    Args:
    company_data - list containing Company_Name, Location, Sector
    
    Returns:
    new_data - dictionary containing formatted data as per columns

    """
    new_data = {}
    columns = ['Company_Name', 'Location', 'Sector']

    for data in company_data:
        subs = re.sub('N/A', '', data).strip() # substitute N/A with blanks
        subs = re.sub('\n-', '-', data).strip() # substitute \n- with -
        splits = subs.split('\n') # split into elements by \n
        stripped = [split.strip() for split in splits if split != ''] 
        for column in columns:
            for element in stripped:  # iterate columns and data to update dictionary
                new_data[column] = element # ERROR-cause: repetition of column name, instead add all values relevant to column in one-go.

    return new_data

In [34]:
trim_data(company_data)

TypeError: expected string or bytes-like object